In [1]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1])
                                                                                             if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset():
    #각 데이터 파일은 공백으로 분리되어 있음
    feature_name_df = pd.read_csv('/content/drive/MyDrive/독학/Python ML/data/human_activity/features.txt', sep=r'\s+', header=None, names=['column_index','column_name'])
    #중복된 피처명을 수정하는 get_new_feature_name_df() 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    #DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    #학습/테스트 피처 데이터를 DataFrame으로 로딩, 칼럼명은 feature_name 적용
    X_train = pd.read_csv('/content/drive/MyDrive/독학/Python ML/data/human_activity/train/X_train.txt', sep=r'\s+', names=feature_name)
    X_test = pd.read_csv('/content/drive/MyDrive/독학/Python ML/data/human_activity/test/X_test.txt', sep=r'\s+', names=feature_name)
    #학습/테스트 레이블 데이터를 DataFrame으로 로딩, 칼럼명은 action으로 부여
    y_train = pd.read_csv('/content/drive/MyDrive/독학/Python ML/data/human_activity/train/y_train.txt', sep=r'\s+', header=None, names=['action'])
    y_test = pd.read_csv('/content/drive/MyDrive/독학/Python ML/data/human_activity/test/y_test.txt', sep=r'\s+', header=None, names=['action'])
    #로딩된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

#GBM 수행 시간 측정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f'GBM 정확도: {gb_accuracy:.4f}')
print(f'GBM 수행 시간: {time.time() - start_time:.1f}')

KeyboardInterrupt: 